In [1]:
!pip install python-Levenshtein


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
from Levenshtein import distance
import os

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Proshir-
[nltk_data]     Pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Proshir-
[nltk_data]     Pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
class UndefinedQueryInputError(Exception):
    def __init__(self):
        self.message = "Undefined query input"

In [40]:
class IRSystem:
    def __init__(self):
        self.documents = []
        self.inverted_index = {}
        self.positional_index = {}
        self.all_cterms = set()
        self.all_kterms = {}
        self.kgram_index = {}

    def prep_correct_col(self):
        folder_path = "docs"
        self.all_cterms = set()
        self.all_kterms = {}
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                document_text = file.read()
                self.add_data_correct(document_text)
        for w_id, word in enumerate(self.all_cterms):
            self.all_kterms[w_id] = word
    
    def preprocess_document(self, words, rm_stop=False, do_stem=False):
        # Remove Stop Words
        if rm_stop:
            stop_words = set(stopwords.words("english"))
            words = [word for word in words if word not in stop_words]
        
        # Stemming
        if do_stem:
            stemmer = PorterStemmer()
            words = [stemmer.stem(word) for word in words]

        return words

    def apreprocess_document(self, document):
        # Tokenization
        words = word_tokenize(document)
    
        # Lowercasing and Remove punctuation
        words = [word.lower() for word in words if word.isalnum()]

        return words

    def bpreprocess_document(self, document, rm_stop=False, do_stem=False):
        words = self.apreprocess_document(document)
        return self.preprocess_document(words, rm_stop, do_stem)
    
    def add_document(self, document):
        document = self.apreprocess_document(document)
        self.documents += [document]

    def add_data_correct(self, document):
        words = self.apreprocess_document(document)
        self.all_cterms |= set(words)
                
    def build_inverted_index(self, rm_stop=True, do_stem=True):
        self.inverted_index = {}
        for doc_id, document in enumerate(self.documents):
            terms = self.preprocess_document(document, rm_stop, do_stem)
            
            for term in terms:
                if term not in self.inverted_index:
                    self.inverted_index[term] = [doc_id]
                elif self.inverted_index[term][-1] != doc_id:
                    self.inverted_index[term] += [doc_id]

    def get_inv_index(self, term):
        return self.inverted_index.get(term, [])
    
    def intersect_sorted(self, so1, so2):
        i, j = 0, 0
        if len(so1) > len(so2): # change order of intersect based on their size (freq)
            so1, so2 = so2, so1
        result = []
        while i < len(so1) and j < len(so2):
            doc1 = so1[i]
            while j < len(so2) and so2[j] < doc1:
                j += 1
            if j == len(so2):
                break
            if doc1 == so2[j]:
                result += [doc1]
            i += 1        
        return result

    def merge_sorted(self, so1, so2):
        i, j = 0, 0
        result = []
        while i < len(so1):
            doc1 = so1[i]
            while j < len(so2) and so2[j] <= doc1:
                if doc1 != so2[j]:
                    result += [so2[j]]
                j += 1
            result += [doc1]
            i += 1
        while j < len(so2):
            result += so2[j]
            j += 1
        return result

    def not_sorted(self, so):
        j = 0
        result = []
        for i in range(len(self.documents)):
            if j == len(so) or so[j] != i:
                result += [i]
            else:
                j += 1
        return result

    def boolean_query(self, query, rm_stop=True, do_stem=True):
        terms = re.findall(r'\b\w+\b', query.lower())
        order = None
        if len(terms) == 3:
            if terms[1] == "and":
                order = "and"
            elif terms[1] == "or":
                order = "or"
            terms = self.bpreprocess_document(f"{terms[0]} {terms[2]}", rm_stop, do_stem) 
        elif len(terms) == 2 and terms[0] == "not":
            order = "not"
            terms = self.bpreprocess_document(terms[1], rm_stop, do_stem) 
        if order == None:
            raise UndefinedQueryInputError()

        if order == "and":
            return self.intersect_sorted(self.get_inv_index(terms[0]), self.get_inv_index(terms[1]))
        elif order == "or":
            return self.merge_sorted(self.get_inv_index(terms[0]), self.get_inv_index(terms[1]))
        else:
            return self.not_sorted(self.get_inv_index(terms[0]))

    def build_positional_index(self, rm_stop=False, do_stem=True):
        self.positional_index = {}
        for doc_id, document in enumerate(self.documents):
            terms = self.preprocess_document(document, rm_stop, do_stem)

            for position, term in enumerate(terms):
                if term not in self.positional_index:
                    self.positional_index[term] = {}
                if doc_id not in self.positional_index[term]:
                    self.positional_index[term][doc_id] = [position]
                else:
                    self.positional_index[term][doc_id] += [position]

    def positional_intersect(self, positions_1, positions_2, kprox):
        answer = []

        idx_1, idx_2 = 0, 0

        while idx_1 < len(positions_1) and idx_2 < len(positions_2):
            pos1 = positions_1[idx_1]
            while idx_2 < len(positions_2) and pos1 - positions_2[idx_2] > kprox:
                idx_2 += 1
            j = idx_2
            if j < len(positions_2) and abs(positions_2[j] - pos1) <= kprox:
                answer += [[pos1, positions_2[j]]]
                j += 1
            idx_1 += 1
        
        return answer        

    def get_pos_index(self, term):
        return self.positional_index.get(term, {})
    
    def proximity_query(self, query, rm_stop=False, do_stem=True):
        terms = re.findall(r'\b\w+\b', query.lower())
        if len(terms) != 4 or terms[1] != 'near':
            raise UndefinedQueryInputError()
        kprox = int(terms[2]) + 1
        terms = self.bpreprocess_document(f"{terms[0]} {terms[3]}", rm_stop, do_stem)
        comm_docs = self.intersect_sorted(list(self.get_pos_index(terms[0]).keys()), list(self.get_pos_index(terms[1]).keys()))
        if not comm_docs:
            return []
        result = []
        for doc_id in comm_docs:
            ans = self.positional_intersect(self.get_pos_index(terms[0])[doc_id], self.get_pos_index(terms[1])[doc_id], kprox)
            if ans:
                result += [doc_id]
            print(doc_id, ans)
        return result

    def correct_spelling(self, query):
        query_terms = self.bpreprocess_document(query, rm_stop=False, do_stem=False)

        corrected_query = []
        for term in query_terms:
            # Find the closest term in the dataset using Levenshtein distance
            closest_term = min(self.all_cterms, key=lambda x: distance(term, x))
            corrected_query.append(closest_term)

        return ' '.join(corrected_query)

    def build_kgram_index(self, k=2):
        self.kgram_index = {}
        for trm_id, cterm in self.all_kterms.items():
            cterm = f"${cterm}$"
            for i in range(len(cterm) - k + 1):
                gram = cterm[i:i+k]
                if gram not in self.kgram_index:
                    self.kgram_index[gram] = [trm_id]
                elif self.kgram_index[gram][-1] != trm_id:
                    self.kgram_index[gram] += [trm_id]

    def get_kgram_index(self, term):
        return self.kgram_index.get(term, [])

    def check_kgram(self, query, word):
        query = query[1:-1]
        for i in range(len(query)):
            if '*' == query[i]:
                star_pos = i
                break
            if word[i] != query[i]:
                return False
        else:
            return True
        j = len(word) - 1
        for i in range(len(query)-1, i, -1):
            if '*' == query[i]:
                if star_pos < i and query[star_pos+1:i-1] not in word[star_pos+1:j]:
                    return False
                break
            if word[j] != query[i]:
                return False
            j -= 1
        return True
    
    def wildcard_query_kgram(self, query, do_post=True, k=2):
        query = query.lower()
        query = f"${query}$"
        grams = set()
        for i in range(len(query) - k + 1):
            if '*' not in query[i:i+k]:
                grams.add(query[i:i+k])
        result = []
        for i, gram in enumerate(grams):
            if not i:
                result = self.get_kgram_index(gram)
            else:
                result = self.intersect_sorted(result, self.get_kgram_index(gram))
            if not result:
                return []
        result = [self.all_kterms[idx] for idx in result]
        if do_post:
            result = [word for word in result if self.check_kgram(query, word)]
        return result

In [41]:
ir_system = IRSystem()

In [7]:
ir_system.add_document("This is a simple example document. It contains several words. The words should be processed.")
ir_system.add_document("Another example document with different content. Spelling correction is important for retrieval")
ir_system.add_document("Another example document to test Boolean search capabilities. This document contains relevant content.")
ir_system.documents

[['this',
  'is',
  'a',
  'simple',
  'example',
  'document',
  'it',
  'contains',
  'several',
  'words',
  'the',
  'words',
  'should',
  'be',
  'processed'],
 ['another',
  'example',
  'document',
  'with',
  'different',
  'content',
  'spelling',
  'correction',
  'is',
  'important',
  'for',
  'retrieval'],
 ['another',
  'example',
  'document',
  'to',
  'test',
  'boolean',
  'search',
  'capabilities',
  'this',
  'document',
  'contains',
  'relevant',
  'content']]

### rm_stop=True, do_stem=True

In [8]:
ir_system.build_inverted_index()
ir_system.inverted_index

{'simpl': [0],
 'exampl': [0, 1, 2],
 'document': [0, 1, 2],
 'contain': [0, 2],
 'sever': [0],
 'word': [0],
 'process': [0],
 'anoth': [1, 2],
 'differ': [1],
 'content': [1, 2],
 'spell': [1],
 'correct': [1],
 'import': [1],
 'retriev': [1],
 'test': [2],
 'boolean': [2],
 'search': [2],
 'capabl': [2],
 'relev': [2]}

### Answers are in 0 base mode.

In [9]:
ir_system.boolean_query("example AND content")

[1, 2]

In [10]:
ir_system.boolean_query("example or simple")

[0, 1, 2]

### It doesn't work because stop words have been removed and "is" is no longer in the query.

In [11]:
ir_system.boolean_query("not indexing")

[0, 1, 2]

In [12]:
ir_system.boolean_query("not content")

[0]

### rm_stop=False, do_stem=True

In [13]:
ir_system.build_positional_index()
ir_system.positional_index

{'thi': {0: [0], 2: [8]},
 'is': {0: [1], 1: [8]},
 'a': {0: [2]},
 'simpl': {0: [3]},
 'exampl': {0: [4], 1: [1], 2: [1]},
 'document': {0: [5], 1: [2], 2: [2, 9]},
 'it': {0: [6]},
 'contain': {0: [7], 2: [10]},
 'sever': {0: [8]},
 'word': {0: [9, 11]},
 'the': {0: [10]},
 'should': {0: [12]},
 'be': {0: [13]},
 'process': {0: [14]},
 'anoth': {1: [0], 2: [0]},
 'with': {1: [3]},
 'differ': {1: [4]},
 'content': {1: [5], 2: [12]},
 'spell': {1: [6]},
 'correct': {1: [7]},
 'import': {1: [9]},
 'for': {1: [10]},
 'retriev': {1: [11]},
 'to': {2: [3]},
 'test': {2: [4]},
 'boolean': {2: [5]},
 'search': {2: [6]},
 'capabl': {2: [7]},
 'relev': {2: [11]}}

In [14]:
ir_system.proximity_query("example NEAR/3 content")

1 [[1, 5]]
2 []


[1]

In [15]:
ir_system.proximity_query("example NEAR/1 test")

2 []


[]

### rm_stop=False, do_stem=True

In [16]:
ir_system.build_inverted_index(rm_stop=False, do_stem=True)
ir_system.inverted_index

{'thi': [0, 2],
 'is': [0, 1],
 'a': [0],
 'simpl': [0],
 'exampl': [0, 1, 2],
 'document': [0, 1, 2],
 'it': [0],
 'contain': [0, 2],
 'sever': [0],
 'word': [0],
 'the': [0],
 'should': [0],
 'be': [0],
 'process': [0],
 'anoth': [1, 2],
 'with': [1],
 'differ': [1],
 'content': [1, 2],
 'spell': [1],
 'correct': [1],
 'import': [1],
 'for': [1],
 'retriev': [1],
 'to': [2],
 'test': [2],
 'boolean': [2],
 'search': [2],
 'capabl': [2],
 'relev': [2]}

In [17]:
ir_system.boolean_query("example AND content", rm_stop=False, do_stem=True)

[1, 2]

In [18]:
ir_system.boolean_query("example or simple", rm_stop=False, do_stem=True)

[0, 1, 2]

In [19]:
ir_system.boolean_query("example or is", rm_stop=False, do_stem=True)

[0, 1, 2]

### rm_stop=True, do_stem=True

In [20]:
ir_system.build_positional_index(rm_stop=True, do_stem=True)
ir_system.positional_index

{'simpl': {0: [0]},
 'exampl': {0: [1], 1: [1], 2: [1]},
 'document': {0: [2], 1: [2], 2: [2, 7]},
 'contain': {0: [3], 2: [8]},
 'sever': {0: [4]},
 'word': {0: [5, 6]},
 'process': {0: [7]},
 'anoth': {1: [0], 2: [0]},
 'differ': {1: [3]},
 'content': {1: [4], 2: [10]},
 'spell': {1: [5]},
 'correct': {1: [6]},
 'import': {1: [7]},
 'retriev': {1: [8]},
 'test': {2: [3]},
 'boolean': {2: [4]},
 'search': {2: [5]},
 'capabl': {2: [6]},
 'relev': {2: [9]}}

### The output is correct, but in fact, the distance between them is calculated less because the word between them is removed.

In [21]:
ir_system.proximity_query("example NEAR/3 content", rm_stop=True, do_stem=True)

1 [[1, 4]]
2 []


[1]

### The answer is wrong because "to", which is a stop word, has been removed, so the distance between these two words has decreased and is acceptable.

In [22]:
ir_system.proximity_query("example NEAR/1 test", rm_stop=True, do_stem=True)

2 [[1, 3]]


[2]

# correct_spelling

In [42]:
ir_system.prep_correct_col()

In [43]:
ir_system.correct_spelling("festivsl funders")

'festival founders'

In [44]:
ir_system.all_cterms

{'clone',
 'done',
 'complaints',
 '11',
 'fixing',
 'and',
 'worked',
 'tires',
 'fill',
 'whatever',
 'inferno',
 'cleared',
 'upset',
 'attendance',
 'all',
 'shave',
 'bob',
 'clicked',
 'ethnic',
 'fines',
 'says',
 'among',
 'gasoline',
 'ran',
 'antennas',
 'aid',
 're',
 'friendly',
 'has',
 'acres',
 'quieter',
 'jail',
 'dropped',
 'irritating',
 'quite',
 'bill',
 'dominic',
 'asked',
 'several',
 'toaster',
 'canine',
 'wrong',
 '50x',
 'venice',
 'facility',
 'reason',
 'walks',
 'eyesight',
 'pianist',
 '15',
 'talk',
 'help',
 'work',
 'vittorios',
 'guy',
 'vehicles',
 'tough',
 'batteries',
 'alan',
 'apply',
 'exact',
 'provide',
 'blocks',
 'driver',
 'homeowner',
 'me',
 'man',
 'cane',
 'convertible',
 'surround',
 'did',
 'week',
 'hamburgers',
 '7',
 'heard',
 'jumped',
 'administration',
 'customers',
 'seemed',
 'respond',
 'recent',
 'delightful',
 'idea',
 'johnson',
 'feet',
 'creek',
 'gunshots',
 'offset',
 'over',
 'lived',
 'drive',
 'lucky',
 'so',
 'do

In [45]:
'nobody' in ir_system.all_cterms

True

In [46]:
ir_system.build_kgram_index()

In [47]:
ir_system.get_kgram_index("no")

[10,
 167,
 212,
 241,
 390,
 632,
 681,
 705,
 776,
 792,
 872,
 950,
 1002,
 1017,
 1021,
 1058,
 1101,
 1163,
 1174]

In [48]:
ir_system.wildcard_query_kgram("nobo*y", False)

['nobody']

In [49]:
ir_system.wildcard_query_kgram("nobody", False)

['nobody']

### We have to do post filter. (nobobody != nobody)

In [50]:
ir_system.wildcard_query_kgram("nobobody", False)

['nobody']

In [51]:
ir_system.check_kgram("$nobo*y$", "nobody")

True

In [52]:
ir_system.wildcard_query_kgram("nobobody")

[]

In [53]:
ir_system.wildcard_query_kgram("nobo*y")

['nobody']

## K=3

In [54]:
ir_system.build_kgram_index(k=3)

In [55]:
ir_system.get_kgram_index("$no")

[390, 632, 681, 705, 792, 950, 1002, 1021, 1163, 1174]

In [56]:
ir_system.wildcard_query_kgram("nobody", False, 3)

['nobody']

In [57]:
ir_system.wildcard_query_kgram("nobobody", False, 3)

[]

In [58]:
ir_system.wildcard_query_kgram("nobody", k=3)

['nobody']